# Linear Regression with PyTorch
This notebook walks through creating a linear regression model using PyTorch with training, testing, and model saving/loading.

In [ ]:
# Step 0: Imports
import torch
from torch import nn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Set device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


In [ ]:
# Step 1: Create synthetic data
# Linear function: y = weight * x + bias
weight = 0.3
bias = 0.9
X = torch.linspace(0, 1, 100).unsqueeze(1)
y = weight * X + bias


In [ ]:
# Step 2: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Send to device
X_train, X_test = X_train.to(device), X_test.to(device)
y_train, y_test = y_train.to(device), y_test.to(device)


In [ ]:
# Step 3: Plot the data
plt.figure(figsize=(8, 5))
plt.scatter(X_train.cpu(), y_train.cpu(), label="Train")
plt.scatter(X_test.cpu(), y_test.cpu(), label="Test")
plt.legend()
plt.title("Train/Test Data")
plt.show()


In [ ]:
# Step 4: Define Model with nn.Module
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(1, requires_grad=True))
        self.bias = nn.Parameter(torch.randn(1, requires_grad=True))

    def forward(self, x):
        return self.weight * x + self.bias

model = LinearRegressionModel().to(device)
print("Initial state_dict:\n", model.state_dict())


In [ ]:
# Step 5: Loss & Optimizer
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)


In [ ]:
# Step 6: Training Loop (300 epochs)
epochs = 300

for epoch in range(epochs):
    model.train()
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        model.eval()
        with torch.inference_mode():
            test_pred = model(X_test)
            test_loss = loss_fn(test_pred, y_test)
        print(f"Epoch {epoch:03} | Train Loss: {loss.item():.4f} | Test Loss: {test_loss.item():.4f}")


In [ ]:
# Step 7: Predictions & Visualization
model.eval()
with torch.inference_mode():
    preds = model(X_test)

# Plot
plt.figure(figsize=(8, 5))
plt.scatter(X_train.cpu(), y_train.cpu(), label="Train")
plt.scatter(X_test.cpu(), y_test.cpu(), label="Test")
plt.scatter(X_test.cpu(), preds.cpu(), label="Predictions", marker="x")
plt.legend()
plt.title("Predictions vs Data")
plt.show()


In [ ]:
# Step 8: Save and Load Model
# Save state_dict
torch.save(model.state_dict(), "linear_regression_model.pth")

# Reload model
loaded_model = LinearRegressionModel().to(device)
loaded_model.load_state_dict(torch.load("linear_regression_model.pth"))
loaded_model.eval()


In [ ]:
# Step 9: Confirm Loaded Model Predictions Match
with torch.inference_mode():
    loaded_preds = loaded_model(X_test)

print("Predictions match:", torch.allclose(preds, loaded_preds))
